# Lorde - Melodrama Project

In [1]:
import bearer
import requests
from bs4 import BeautifulSoup

In [2]:
base_url = "https://api.genius.com"
headers = {'Authorization': bearer.TOKEN}

In [3]:
def lyrics_from_song_api_path(song_api_path):
  song_url = base_url + song_api_path
  response = requests.get(song_url, headers=headers)
  json = response.json()
  path = json["response"]["song"]["path"]
  #gotta go regular html scraping... come on Genius
  page_url = "http://genius.com" + path
  page = requests.get(page_url)
  html = BeautifulSoup(page.text, "html.parser")
  #remove script tags that they put in the middle of the lyrics
  [h.extract() for h in html('script')]
  #at least Genius is nice and has a tag called 'lyrics'!
  lyrics = html.find("div", class_='lyrics').get_text() #updated css where the lyrics are based in HTML
  return lyrics

In [4]:
album_url = 'https://genius.com/albums/Lorde/Melodrama'
artist_name = 'Lorde'

In [6]:
page = requests.get(album_url)
track_soup = BeautifulSoup(page.text, 'lxml')
tracks = [i.get_text('*', strip=True) for i in track_soup.find_all('h3', class_='chart_row-content-title')]
tracks = tracks[:len(tracks)-1]
for i in range(len(tracks)):
  tracks[i] = tracks[i].split('*')[0]
tracks

[u'Green Light',
 u'Sober',
 u'Homemade Dynamite',
 u'The Louvre',
 u'Liability',
 u'Hard Feelings/Loveless',
 u'Sober II (Melodrama)',
 u'Writer in the Dark',
 u'Supercut',
 u'Liability (Reprise)',
 u'Perfect Places']

In [7]:
search_url = base_url + "/search"
album_lyrics = {}
for song_title in tracks:
  data = {'q': song_title}
  response = requests.get(search_url, data=data, headers=headers)
  json = response.json()
  song_info = None
  for hit in json["response"]["hits"]:
    if hit["result"]["primary_artist"]["name"] == artist_name:
      song_info = hit
      break
  if song_info:
    song_api_path = song_info["result"]["api_path"]
    album_lyrics[song_title] = lyrics_from_song_api_path(song_api_path)

In [25]:
# This is will create a local instance of the PyDictionary class and now it can be used to get meanings, translations etc.
from PyDictionary import PyDictionary
dictionary=PyDictionary()

In [12]:
# sort each lyric into their parts of speech
parts_of_speech = {}

In [24]:
for i in album_lyrics.keys():
  album_lyrics[i] = album_lyrics[i].split()

In [21]:
# filter by part of speech
for i in album_lyrics.keys():
  for j in album_lyrics[i]:
      